In [ ]:
from torchvision import datasets, transforms
import torch
from __future__ import print_function
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import os
import math
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from random import sample
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/Deep learning

Dataloader

In [ ]:
class OfficeHomeDataset(Dataset):
    def __init__(self, data_path, domain="Real World", balance=False, one_hot=False, transform=None):
        self.transform = transform
        self.domain = domain
        self.balance = balance
        self.one_hot = one_hot

        # label dict
        self.label_dict = {"Art": 0, "Clipart":1, "Product":2, 'Real World': 3}

        # Read all file names
        self.file_names = []
        if self.domain is None:
            self.n_classes = 3
            for root, dirs, files in os.walk(data_path):
                for filename in files:
                    if filename == ".DS_Store": continue
                    elif os.path.splitext(filename)[-1] == ".txt": continue
                    self.file_names.append(os.path.join(root, filename))
        else:
            self.n_classes = 2
            domain_file = []
            source_file = []
            for root, dirs, files in os.walk(data_path):
                if self.domain in root:
                    for filename in files:
                        if filename == ".DS_Store": continue
                        elif os.path.splitext(filename)[-1] == ".txt": continue
                        domain_file.append(os.path.join(root, filename))
                else:
                    for filename in files:
                        if filename == ".DS_Store": continue
                        elif os.path.splitext(filename)[-1] == ".txt": continue
                        source_file.append(os.path.join(root, filename))
            if balance:
                self.file_names = domain_file + sample(source_file, len(domain_file))
            else:
                self.file_names = domain_file + source_file
        
        print(len(self.file_names))
        # self.file_names = sample(self.file_names, 200)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = []
        filename = self.file_names[idx]
        img = Image.open(filename)
        if self.transform:
            img = self.transform(img)
        # print(img.shape, filename)
        source_name = filename.split('/')[-3]
        if self.domain is None:
            label.append(self.label_dict[source_name])
        else:
            if source_name == self.domain:
                label.append(1)
            else: label.append(0)
        if self.one_hot:
            label = np.array(label)
            label = np.eye(self.n_classes)[label]
            label = np.float32(label)
        else:
            label = np.array(label)
        # sample = {'image': img, 'label': label}
        sample = [img, label]

        return sample

In [ ]:
def load_training(root_path, dir, batch_size, kwargs):
    transform = transforms.Compose(
        [transforms.Resize([256, 256]),
         transforms.RandomCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor()])
    data = OfficeHomeDataset(os.path.join(root_path, dir), transform=transform)
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs)
    return train_loader

def load_testing(root_path, dir, batch_size, kwargs):
    transform = transforms.Compose(
        [transforms.Resize([224, 224]),
         transforms.ToTensor()])
    data = OfficeHomeDataset(os.path.join(root_path, dir), transform=transform)
    test_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, **kwargs)
    return test_loader

models

# source classifier

In [ ]:
class SourceClassifer(nn.Module):
    def __init__(self, f_dim=256, n_classes=65):
        super(SourceClassifer, self).__init__()

        self.f_dim = f_dim
        self.n_classes = n_classes

        # Get ResNet50 model
        ResNet50 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=False)
        ResNet50.fc = nn.Identity()
        self.ResNet50 = ResNet50

        self.extractor1 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.f_dim),
            nn.ELU(),
            nn.Linear(self.f_dim, self.f_dim),
            nn.BatchNorm1d(self.f_dim),
            nn.ELU()
        )

        self.extractor2 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.f_dim),
            nn.ELU(),
            nn.Linear(self.f_dim, self.f_dim),
            nn.BatchNorm1d(self.f_dim),
            nn.ELU()
        )

        self.extractor3 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.f_dim),
            nn.ELU(),
            nn.Linear(self.f_dim, self.f_dim),
            nn.BatchNorm1d(self.f_dim),
            nn.ELU()
        )

        self.cls1 = nn.Linear(self.f_dim, self.n_classes)
        self.cls2 = nn.Linear(self.f_dim, self.n_classes)
        self.cls3 = nn.Linear(self.f_dim, self.n_classes)

    def forward(self, data_src, label_src = 0, mark = 1, training=True):
        
        if training == True:
            h1 = self.ResNet50(data_src)
            h1 = torch.flatten(h1, start_dim=1)  # size: (batch_size, dim)

            if mark == 1:
                feature1 = self.extractor1(h1)
                pred1 = self.cls1(feature1)

                cls_loss = F.cross_entropy(pred1, label_src)

                return cls_loss

            if mark == 2:
                feature2 = self.extractor2(h1)
                pred2 = self.cls2(feature2)

                cls_loss = F.cross_entropy(pred2, label_src)

                return cls_loss

            if mark == 3:
                feature3 = self.extractor3(h1)
                pred3 = self.cls3(feature3)

                cls_loss = F.cross_entropy(pred3, label_src)

                return cls_loss

        else:
            h1 = self.ResNet50(data_src)
            h1 = torch.flatten(h1, start_dim=1)  # size: (batch_size, dim)

            feature1 = self.extractor1(h1)
            pred1 = self.cls1(feature1)

            feature2 = self.extractor2(h1)
            pred2 = self.cls2(feature2)

            feature3 = self.extractor3(h1)
            pred3 = self.cls3(feature3)

            return pred1, pred2, pred3, feature1, feature2, feature3


# domain classifier

In [ ]:
class DomainClassifier(nn.Module):
  def __init__(self, sourceClassifier, f_dim=256, n_classes=2):
    super(DomainClassifier, self).__init__()
    self.f_dim = f_dim
    self.half_f_dim = self.f_dim // 2
    self.n_classes = n_classes

    self.sourceClassifier = sourceClassifier

    self.domain_cls1 = nn.Sequential(
        nn.Linear(self.f_dim, self.half_f_dim),
        nn.ELU(),
        nn.Linear(self.half_f_dim, self.n_classes)
    )

    self.domain_cls2 = nn.Sequential(
        nn.Linear(self.f_dim, self.half_f_dim),
        nn.ELU(),
        nn.Linear(self.half_f_dim, self.n_classes)
    )

    self.domain_cls3 = nn.Sequential(
        nn.Linear(self.f_dim, self.half_f_dim),
        nn.ELU(),
        nn.Linear(self.half_f_dim, self.n_classes)
    )

  def forward(self, data_src, data_tgt=None, label_src=0, label_tgt=0, mark=1, training=True):

    if training == True:
        _, _, _, feature1, feature2, feature3 = self.sourceClassifier(data_src, training=False)
        _, _, _, feature1_tgt, feature2_tgt, feature3_tgt = self.sourceClassifier(data_tgt, training=False)

        if mark == 1:
            logits1 = self.domain_cls1(feature1)
            logits1_tgt = self.domain_cls1(feature1_tgt)
            a = 1 / data_src.shape[0]
            weights = torch.Tensor([a] * self.n_classes).to(device)

            cls_loss = F.cross_entropy(logits1, label_src, weight=weights) \
                + F.cross_entropy(logits1_tgt, label_tgt, weight=weights)

            return cls_loss

        if mark == 2:
            logits2 = self.domain_cls2(feature2)
            logits2_tgt = self.domain_cls2(feature2_tgt)
            a = 1 / data_src.shape[0]
            weights = torch.Tensor([a] * self.n_classes).to(device)

            cls_loss = F.cross_entropy(logits2, label_src, weight=weights) \
                + F.cross_entropy(logits2_tgt, label_tgt, weight=weights)

            return cls_loss

        if mark == 3:
            logits3 = self.domain_cls1(feature3)
            logits3_tgt = self.domain_cls1(feature3_tgt)
            a = 1 / data_src.shape[0]
            weights = torch.Tensor([a] * self.n_classes).to(device)

            cls_loss = F.cross_entropy(logits3, label_src, weight=weights) \
                + F.cross_entropy(logits3_tgt, label_tgt, weight=weights)

            return cls_loss

    else:
        _, _, _, feature1, feature2, feature3 = self.sourceClassifier(data_src, training=False)

        logits1 = self.domain_cls1(feature1)
        logits2 = self.domain_cls2(feature2)
        logits3 = self.domain_cls3(feature3)

        return logits1, logits2, logits3


## train and test function

In [ ]:
batch_size = 16
iteration = 6000 // 16
epoch = 3
cuda = True
seed = 8
log_interval = 20
class_num = 65
root_path = "./Dataset/"
source1_name = "Art"
source2_name = 'Clipart'
source3_name = 'Product'
target_name = "Real World"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

def train(model):
    source1_loader = load_training(root_path, source1_name, batch_size, kwargs)
    source2_loader = load_training(root_path, source2_name, batch_size, kwargs)
    source3_loader = load_training(root_path, source3_name, batch_size, kwargs)

    target_loader = load_training(root_path, target_name, batch_size, kwargs)
    
    source1_iter = iter(source1_loader)
    source2_iter = iter(source2_loader)
    source3_iter = iter(source3_loader)
    target_iter = iter(target_loader)

    for i in range(1, iteration + 1):
        model.train()        
        LEARNING_RATE = 1e-4
        optimizer = torch.optim.Adam([
            {'params': model.domain_cls1.parameters(), 'lr': LEARNING_RATE},
            {'params': model.domain_cls2.parameters(), 'lr': LEARNING_RATE},
            {'params': model.domain_cls3.parameters(), 'lr': LEARNING_RATE}
        ])

        try:
            source_data, source_label = source1_iter.next()
        except Exception as err:
            source1_iter = iter(source1_loader)
            source_data, source_label = source1_iter.next()
        
        try:
            target_data, target_label = target_iter.next()
        except Exception as err:
            target_iter = iter(target_loader)
            target_data, target_label = target_iter.next()

        if cuda:
            source_data, source_label = source_data.cuda(), source_label.reshape(-1).cuda()
            target_data, target_label = target_data.cuda(), target_label.reshape(-1).cuda()
            
        source_data, source_label = Variable(source_data), Variable(source_label)
        target_data, target_label = Variable(target_data), Variable(target_label)
        optimizer.zero_grad()

        cls_loss = model(source_data, target_data, source_label, target_label, mark=1)
        loss = cls_loss
        loss.backward()
        optimizer.step()

        if i % log_interval == 0:
            print('Train source1 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))

        try:
            source_data, source_label = source2_iter.next()
        except Exception as err:
            source2_iter = iter(source2_loader)
            source_data, source_label = source2_iter.next()
      
        try:
            target_data, target_label = target_iter.next()
        except Exception as err:
            target_iter = iter(target_loader)
            target_data, target_label = target_iter.next()

        if cuda:
            source_data, source_label = source_data.cuda(), source_label.reshape(-1).cuda()
            target_data, target_label = target_data.cuda(), target_label.reshape(-1).cuda()

        source_data, source_label = Variable(source_data), Variable(source_label)
        target_data, target_label = Variable(target_data), Variable(target_label)
        optimizer.zero_grad()

        cls_loss = model(source_data, target_data, source_label, target_label, mark=2)
        loss = cls_loss
        loss.backward()
        optimizer.step()

        if i % log_interval == 0:
            print('Train source2 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))

        try:
            source_data, source_label = source3_iter.next()
        except Exception as err:
            source3_iter = iter(source3_loader)
            source_data, source_label = source3_iter.next()

        try:
            target_data, target_label = target_iter.next()
        except Exception as err:
            target_iter = iter(target_loader)
            target_data, target_label = target_iter.next()

        if cuda:
            source_data, source_label = source_data.cuda(), source_label.reshape(-1).cuda()
            target_data, target_label = target_data.cuda(), target_label.reshape(-1).cuda()

        source_data, source_label = Variable(source_data), Variable(source_label)
        target_data, target_label = Variable(target_data), Variable(target_label)
        optimizer.zero_grad()

        cls_loss = model(source_data, target_data, source_label, target_label, mark=3)
        loss = cls_loss
        loss.backward()
        optimizer.step()

        if i % log_interval == 0:
            print('Train source3 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))
            
    return model

def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    correct1 = 0
    correct2 = 0
    correct3 = 0

    target_test_loader = load_testing(root_path, target_name, batch_size, kwargs)

    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.reshape(-1).cuda()
            data, target = Variable(data), Variable(target)
            pred1, pred2, pred3 = model(data, training=False)

            # pred1 = torch.nn.functional.softmax(pred1, dim=1)
            # pred2 = torch.nn.functional.softmax(pred2, dim=1)
            # pred3 = torch.nn.functional.softmax(pred3, dim=1)

            pred = (pred1 + pred2 + pred3) / 3
            test_loss += F.cross_entropy(pred, target).item()  # sum up batch loss
            pred = pred.data.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred1.data.max(1)[1]  # get the index of the max log-probability
            correct1 += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred2.data.max(1)[1]  # get the index of the max log-probability
            correct2 += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred3.data.max(1)[1]  # get the index of the max log-probability
            correct3 += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(target_test_loader.dataset)
        print(target_name, '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(target_test_loader.dataset),
            100. * correct / len(target_test_loader.dataset)))
        print('\nsource1 accnum {}, source2 accnum {}，source3 accnum {}'.format(correct1, correct2, correct3))
    return correct

if __name__ == '__main__':
    sourceClassifier = SourceClassifer(n_classes=class_num)
    sourceClassifier.load_state_dict(torch.load("./results/source_classifer_ACP"))
    model = DomainClassifier(sourceClassifier, n_classes=2)
    best_model_wts = copy.deepcopy(model.state_dict())
    if cuda:
        model.cuda()
    correct = 0
    for _ in range(epoch):
        model = train(model)
        t_correct = test(model)
        if t_correct > correct:
            correct = t_correct
            best_model_wts = copy.deepcopy(model.state_dict())
        print(source1_name, source2_name, source3_name, "with", target_name, "%s max correct:" % target_name, correct.item(), "\n")
    torch.save(best_model_wts, f"./results/domain_classifer_ACP")

# weight calculation

In [ ]:
def get_files_length(domain):
  file_names = []
  data_path = os.path.join(root_path, domain)
  print(data_path)
  for root, dirs, files in os.walk(data_path):
      for filename in files:
          if filename == ".DS_Store": continue
          elif os.path.splitext(filename)[-1] == ".txt": continue
          file_names.append(os.path.join(root, filename))
  return(len(file_names))

batch_size = 32

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda = True
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
root_path = "./Dataset"
sources = ["Art", "Clipart", "Product"]
lens = {}
for source in sources:
  lens[source] = get_files_length(source)
print(lens)
target = "Real World"
target_length = get_files_length(target)
# sourceClassifier = SourceClassifer()
# # sourceClassifier.load_state_dict(torch.load("./trained_models/source_classifer"))
# domainClassifier = DomainClassifier(sourceClassifier)
# # domainClassifier.load_state_dict(torch.load("./trained_models/domain_classifier_Real World"))
# domainClassifier.to(device)

domainClassifier = model

hi = []
for i, source in enumerate(sources):
  dataloader = load_training(root_path, source, batch_size, kwargs)
    
  running_corrects = 0
  correct_one = []
  correct_zero = []
  for data, target in dataloader:
        if cuda:
            data, target = data.cuda(), target.reshape(-1).cuda()
        data, target = Variable(data), Variable(target)
        logits1, logits2, logits3 = domainClassifier(data, training=False)

        if i == 0:
            _, preds = torch.max(logits1, 1)
        elif i == 1:
            _, preds = torch.max(logits2, 1)
        elif i == 2:
            _, preds = torch.max(logits3, 1)

        correct_labbels = torch.where(preds == target, preds, -1)
        correct_one.append((correct_labbels == 1).sum())
        correct_zero.append((correct_labbels == 0).sum())
        # print(preds)
        # print(target)
        # print(labels.shape, (preds == labels).sum(), (correct_labbels == 1).sum(), (correct_labbels == 0).sum())

        # statistics
        running_corrects += torch.sum(preds == target.data)
  
  all_ones = sum(correct_one)
  all_zeros = sum(correct_zero)
  print("source: ", all_ones, " target: ", all_zeros)
  tmp = abs(all_zeros/target_length - all_ones/lens[source])
  print(tmp)
  hi.append(tmp)
  epoch_acc = running_corrects.double() / len(dataloader.dataset)
  print(running_corrects, len(dataloader.dataset))
  print('Acc: {:.4f}'.format(epoch_acc))

print(hi)

In [ ]:
print("hi: ", hi)

K = 3
hi = torch.Tensor(hi)
alpha = torch.exp(-hi * K) / torch.sum(torch.exp(-hi * K))

print("alpha: ", alpha)

In [ ]:
K = 3
alpha = [0.5580, 0.2512, 0.1907]

In [ ]:
beta = {}

for i in range(K):
  for j in range(i, K):
    k = f"{i}"+"-"+f"{j}"
    beta[k] = min(alpha[i], alpha[j])

beta

# weighted aligned

## loss

In [ ]:
def loss_qt(output, target, mark=1, relevance=alpha, n_classes=3):
  label = target[0]
  weight = relevance[mark-1] / output.shape[0]

  loss = F.cross_entropy(output, target) * weight
  return loss

In [ ]:
def CORAL(source, target):
    d = source.data.shape[1]

    # source covariance
    xm = torch.mean(source, 0, keepdim=True) - source
    xc = xm.t() @ xm

    # target covariance
    xmt = torch.mean(target, 0, keepdim=True) - target
    xct = xmt.t() @ xmt

    # frobenius norm between source and target
    loss = torch.mean(torch.mul((xc - xct), (xc - xct)))
    loss = loss/(4*d*d)

    return loss

In [ ]:
def coral(source, target):
    if len(source.shape) == 1:
      d = source.shape[0]
    else:
      d = source.shape[1]  # dim vector

    source_c = compute_covariance(source)
    target_c = compute_covariance(target)

    loss = torch.sum(torch.mul((source_c - target_c), (source_c - target_c)))

    loss = loss / (4 * d * d)
    return loss


def compute_covariance(input_data):
    """
    Compute Covariance matrix of the input data
    """
    n = input_data.shape[0]  # batch_size

    # Check if using gpu or cpu
    if input_data.is_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    id_row = torch.ones(n).resize(1, n).to(device=device)
    sum_column = torch.mm(id_row, input_data)
    mean_column = torch.div(sum_column, n)
    term_mul_2 = torch.mm(mean_column.t(), mean_column)
    d_t_d = torch.mm(input_data.t(), input_data)
    c = torch.add(d_t_d, (-1 * term_mul_2)) * 1 / (n - 1)

    return c

In [ ]:
def loss_align(source1_output, source2_output, source3_output, target_output, K=3, alpha=alpha, beta=beta):
  loss = 0.0

  loss += alpha[0] * CORAL(source1_output, target_output)
  loss += alpha[1] * CORAL(source2_output, target_output)
  loss += alpha[2] * CORAL(source3_output, target_output)

  loss += beta["0-1"] * CORAL(source1_output, source2_output) / (K-1)
  loss += beta["0-2"] * CORAL(source1_output, source3_output) / (K-1)
  loss += beta["1-2"] * CORAL(source2_output, source3_output) / (K-1)

  return loss

## weighted aligned source encoder, target classifier

In [ ]:
class Targetclassifier(nn.Module):
    def __init__(self, sourceClassifier, f_dim=256, c_dim=256, n_classes=65):
        super(Targetclassifier, self).__init__()

        self.sourceClassifier = sourceClassifier
        self.f_dim = f_dim
        self.c_dim = c_dim
        self.n_classes = n_classes

        self.encoder1 = nn.Sequential(
            nn.Linear(self.f_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU(),
            nn.Linear(self.c_dim, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU()
        )

        self.encoder2 = nn.Sequential(
            nn.Linear(self.f_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU(),
            nn.Linear(self.c_dim, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU()
        )

        self.encoder3 = nn.Sequential(
            nn.Linear(self.f_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU(),
            nn.Linear(self.c_dim, self.c_dim),
            nn.BatchNorm1d(self.c_dim),
            nn.ELU()
        )

        self.cls = nn.Sequential(
            nn.Linear(self.c_dim, self.c_dim),
            nn.ELU(),
            nn.Linear(self.c_dim, self.n_classes)
        )

    def forward(self, data_src, label_src=0, mark=1, training=True, encoding=None):

        if training == True:

            if mark == 1:
                _, _, _, source_feature, _, _ = self.sourceClassifier(data_src, training=False)
                feature1 = self.encoder1(source_feature)
                pred1 = self.cls(feature1)

                loss = loss_qt(pred1, label_src, mark=1)

                return loss

            if mark == 2:
                _, _, _, _, source_feature, _ = self.sourceClassifier(data_src, training=False)
                feature2 = self.encoder2(source_feature)
                pred2 = self.cls(feature2)

                loss = loss_qt(pred2, label_src, mark=2)

                return loss

            if mark == 3:
                _, _, _, _, _, source_feature = self.sourceClassifier(data_src, training=False)
                feature3 = self.encoder3(source_feature)
                pred3 = self.cls(feature3)

                loss = loss_qt(pred3, label_src, mark=3)

                return loss

        else:
            if encoding is not None:
                pred = self.cls(encoding)

                return pred

            else:

                _, _, _, source_feature, _, _ = self.sourceClassifier(data_src, training=False)
                feature1 = self.encoder1(source_feature)
                pred1 = self.cls(feature1)

                _, _, _, _, source_feature, _ = self.sourceClassifier(data_src, training=False)
                feature2 = self.encoder2(source_feature)
                pred2 = self.cls(feature2)

                _, _, _, _, _, source_feature = self.sourceClassifier(data_src, training=False)
                feature3 = self.encoder3(source_feature)
                pred3 = self.cls(feature3)

                return pred1, pred2, pred3, feature1, feature2, feature3


## weighted aligned target encoder

In [ ]:
class TargetEncoder(nn.Module):
    def __init__(self, f_dim=256, n_classes=65):
        super(TargetEncoder, self).__init__()
        self.f_dim = f_dim
        self.n_classes = n_classes

        # Get ResNet50 model
        ResNet50 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
        ResNet50.fc = nn.Identity()
        self.ResNet50 = ResNet50
        for param in self.ResNet50.parameters():
            param.requires_grad = False

        self.encoder = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ELU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),  # expect 2-D input
            nn.ELU(),
            nn.Linear(1024, self.f_dim),
            nn.ELU(),
            nn.Linear(self.f_dim, self.f_dim),
            nn.BatchNorm1d(self.f_dim),
            nn.ELU()
        )

    def forward(self, input_batch):
        h1 = self.ResNet50(input_batch)
        h1 = torch.flatten(h1, start_dim=1)  # size: (batch_size, dim)
        feature = self.encoder(h1)
        return feature

## training

## data loader

In [ ]:
def load_training(root_path, dir, batch_size, kwargs):
    transform = transforms.Compose(
        [transforms.Resize([256, 256]),
         transforms.RandomCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor()])
    data = datasets.ImageFolder(root=os.path.join(root_path, dir), transform=transform)
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs)
    return train_loader

def load_testing(root_path, dir, batch_size, kwargs):
    transform = transforms.Compose(
        [transforms.Resize([224, 224]),
         transforms.ToTensor()])
    data = datasets.ImageFolder(root=os.path.join(root_path, dir), transform=transform)
    test_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, **kwargs)
    return test_loader

In [ ]:
batch_size = 16
iteration = 6000 // 16
epoch = 3
cuda = True
seed = 8
log_interval = 20
class_num = 65
root_path = "./Dataset/"
source1_name = "Art"
source2_name = 'Clipart'
source3_name = 'Product'
target_name = "Real World"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

def train(model1, model2):
    source1_loader = load_training(root_path, source1_name, batch_size, kwargs)
    source2_loader = load_training(root_path, source2_name, batch_size, kwargs)
    source3_loader = load_training(root_path, source3_name, batch_size, kwargs)
    target_loader = load_training(root_path, target_name, batch_size, kwargs)
    
    source1_iter = iter(source1_loader)
    source2_iter = iter(source2_loader)
    source3_iter = iter(source3_loader)
    target_iter = iter(target_loader)

    for i in range(1, iteration + 1):
        model1.train()
        model2.train()          
        LEARNING_RATE = 1e-4

        # optimizer of source encoder & target classifier
        optimizer1 = torch.optim.Adam([
            {'params': model1.encoder1.parameters(), 'lr': LEARNING_RATE},
            {'params': model1.encoder2.parameters(), 'lr': LEARNING_RATE},
            {'params': model1.encoder3.parameters(), 'lr': LEARNING_RATE},
            {'params': model1.cls.parameters(), 'lr': LEARNING_RATE}
        ])

        # optimizer of target encoder
        optimizer2 = torch.optim.Adam([
            {'params': model2.encoder.parameters(), 'lr': LEARNING_RATE}
        ])

        try:
            source1_data, source1_label = source1_iter.next()
        except Exception as err:
            source1_iter = iter(source1_loader)
            source1_data, source1_label = source1_iter.next()
          
        source1_data, source1_label = source1_data.to(device), source1_label.reshape(-1).to(device)
        source1_data, source1_label = Variable(source1_data), Variable(source1_label)
        optimizer1.zero_grad()

        qt_loss = model1(source1_data, source1_label, mark=1)
        loss = qt_loss
        loss.backward()
        optimizer1.step()

        if i % log_interval == 0:
            print('Train source1 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))

        try:
            source2_data, source2_label = source2_iter.next()
        except Exception as err:
            source2_iter = iter(source2_loader)
            source2_data, source2_label = source2_iter.next()
          
        source2_data, source2_label = source2_data.to(device), source2_label.reshape(-1).to(device)
        source2_data, source2_label = Variable(source2_data), Variable(source2_label)
        optimizer1.zero_grad()

        qt_loss = model1(source2_data, source2_label, mark=1)
        loss = qt_loss
        loss.backward()
        optimizer1.step()

        if i % log_interval == 0:
            print('Train source2 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))

        try:
            source3_data, source3_label = source3_iter.next()
        except Exception as err:
            source3_iter = iter(source3_loader)
            source3_data, source3_label = source3_iter.next()
          
        source3_data, source3_label = source3_data.to(device), source3_label.reshape(-1).to(device)
        source3_data, source3_label = Variable(source3_data), Variable(source3_label)
        optimizer1.zero_grad()

        qt_loss = model1(source3_data, source3_label, mark=1)
        loss = qt_loss
        loss.backward()
        optimizer1.step()

        if i % log_interval == 0:
            print('Train source3 iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item()))
            
        try:
            target_data, __ = target_iter.next()
        except Exception as err:
            target_iter = iter(target_loader)
            target_data, __ = target_iter.next()

        target_data = target_data.to(device)
        target_data = Variable(target_data)
        optimizer2.zero_grad()

        target_feature = model2(target_data)

        _, _, _, feature1, _, _ = model1(source1_data, training=False)
        _, _, _, _, feature2, _ = model1(source2_data, training=False)
        _, _, _, _, _, feature3 = model1(source3_data, training=False)
        loss2 = loss_align(feature1, feature2, feature3, target_feature)
        loss2.backward()
        optimizer2.step()

        if i % log_interval == 0:
              print('Train target iter: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                  i, 100. * i / iteration, loss2.item()))
    
    return model1, model2

def test(mode11, model2):
    model1.eval()
    model2.eval()

    test_loss = 0
    correct = 0
    correct1 = 0
    correct2 = 0
    correct3 = 0

    target_test_loader = load_testing(root_path, target_name, batch_size, kwargs)

    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.reshape(-1).cuda()
            data, target = Variable(data), Variable(target)
            pred1, pred2, pred3, _, _, _ = model1(data, training=False)

            # pred1 = torch.nn.functional.softmax(pred1, dim=1)
            # pred2 = torch.nn.functional.softmax(pred2, dim=1)
            # pred3 = torch.nn.functional.softmax(pred3, dim=1)

            pred = (pred1 + pred2 + pred3) / 3
            test_loss += F.cross_entropy(pred, target).item()  # sum up batch loss
            pred = pred.data.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred1.data.max(1)[1]  # get the index of the max log-probability
            correct1 += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred2.data.max(1)[1]  # get the index of the max log-probability
            correct2 += pred.eq(target.data.view_as(pred)).cpu().sum()
            pred = pred3.data.max(1)[1]  # get the index of the max log-probability
            correct3 += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(target_test_loader.dataset)
        print(target_name, '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(target_test_loader.dataset),
            100. * correct / len(target_test_loader.dataset)))
        print('\nsource1 accnum {}, source2 accnum {}，source3 accnum {}'.format(correct1, correct2, correct3))
    return correct

if __name__ == '__main__':
    sourceClassifier = SourceClassifer(n_classes=class_num)
    sourceClassifier.load_state_dict(torch.load("./results/source_classifer_ACP"))
    for param in sourceClassifier.ResNet50.parameters():
        param.requires_grad = False

    model1 = Targetclassifier(sourceClassifier)
    model2 = TargetEncoder()
    
    best_model_wts1 = copy.deepcopy(model1.state_dict())
    best_model_wts2 = copy.deepcopy(model2.state_dict())

    if cuda:
        model1.cuda()
        model2.cuda()
    correct = 0
    for _ in range(epoch):
        model1, model2 = train(model1, model2)
        t_correct = test(model1, model2)
        if t_correct > correct:
            correct = t_correct
            best_model_wts = copy.deepcopy(model1.state_dict())
        print(source1_name, source2_name, source3_name, "with", target_name, "%s max correct:" % target_name, correct.item(), "\n")
    best_model_wts2 = copy.deepcopy(model2.state_dict())
    torch.save(best_model_wts1, f"./results/target_classifer_ACP")
    torch.save(best_model_wts2, f"./results/target_encoder_ACP")  

# loss T->W

## weights

In [ ]:
batch_size = 16
iteration = 6000 // 16
epoch = 1
cuda = True
seed = 8
log_interval = 20
class_num = 65
root_path = "./Dataset/"
source1_name = "Art"
source2_name = 'Clipart'
source3_name = 'Product'
target_name = "Real World"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

sourceClassifier = SourceClassifer(n_classes=class_num)
sourceClassifier.load_state_dict(torch.load("./results/source_classifer_ACP"))
model = DomainClassifier(sourceClassifier, n_classes=2)
model.load_state_dict(torch.load("./results/domain_classifer_ACP"))
softmax = nn.Softmax(dim=1)

if cuda:
    model.cuda()

target_test_loader = load_testing(root_path, target_name, batch_size, kwargs)
source1_weight = []
source2_weight = []
source3_weight = []

with torch.no_grad():
    for data, target in target_test_loader:
        if cuda:
            data, target = data.cuda(), target.reshape(-1).cuda()
        data, target = Variable(data), Variable(target)
        logits1, logits2, logits3 = model(data, training=False)
        logits1 = softmax(logits1)
        logits2 = softmax(logits2)
        logits3 = softmax(logits3)

        weights_sum = logits1[:, 1] * alpha[0] + logits2[:, 1] * alpha[1] + logits3[:, 1] * alpha[2]

        w1 = logits1[:, 1] * alpha[0] / weights_sum
        w2 = logits2[:, 1] * alpha[1] / weights_sum
        w3 = logits3[:, 1] * alpha[2] / weights_sum

        source1_weight.append(w1)
        source2_weight.append(w2)
        source3_weight.append(w3)

In [ ]:
len(source2_weight)

## loss

In [ ]:
def loss_tw(es, et, w, ind, mark=1):
    res = 0.0

    loss = nn.MSELoss()
    res = torch.sum(w * loss(et, es))

    return res

In [ ]:
input = torch.randn(16, 8, requires_grad=True)
target = torch.randn(16, 8)
loss = loss_tw(input, target, 0)
print(loss)

In [ ]:
def loss_distill(w1, w2, w3, pred1, pred2, pred3, pred):
    loss = nn.L1Loss()

    res = 0.0
    w1 = w1.reshape(-1, 1)
    w2 = w2.reshape(-1, 1)
    w3 = w3.reshape(-1, 1)

    phi = w1 * pred1 + w2 * pred2 + w3 * pred3

    res = loss(pred, phi)

    return res

In [ ]:
input = torch.randn(16, 8, requires_grad=True)
target = torch.randn(16, 8)
w = torch.randn(16)
loss = loss_distill(w, w, w, input, input, input, target)
print(loss)

In [ ]:
def loss_entropy(pred):
    res = 0.0
    p = F.softmax(pred, dim=-1)
    res = -1 * torch.sum(p * F.log_softmax(pred, dim=-1)) / pred.size()[0]

    return res

In [ ]:
def loss_de(iteration, distill, entropy, m=0.0036):
    res = 0.0
    mu = min(1, m*iteration)

    res = (1-mu) * distill + mu * entropy

    return res

  ## training

In [ ]:
iteration = len(source1_weight)
print("iteration: ", iteration)
epoch = 15
cuda = True
seed = 8
log_interval = 20
class_num = 65
batch_size = 32
root_path = "./Dataset/"
source1_name = "Art"
source2_name = 'Clipart'
source3_name = 'Product'
target_name = "Real World"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

softmax = nn.Softmax(dim=1)

target_test_loader = load_testing(root_path, target_name, batch_size, kwargs)

def train(model1, model2, domainclassifier):

    # target_test_loader = load_training(root_path, target_name, batch_size, kwargs)
    target_iter = iter(target_test_loader)  

    for param in model1.parameters():
        param.requires_grad = False

    for i in range(1, iteration + 1):
        model1.eval()
        model2.train()         
        LEARNING_RATE = 1e-4

        # optimizer of target encoder
        optimizer2 = torch.optim.Adam([
            {'params': model2.encoder.parameters(), 'lr': LEARNING_RATE}
        ])

        try:
            target_data, __ = target_iter.next()
        except Exception as err:
            target_iter = iter(target_test_loader)
            target_data, __ = target_iter.next()

        target_data = target_data.to(device)
        target_data = Variable(target_data)

        pred1, pred2, pred3 = domainclassifier(target_data, training=False)
        sc_pred1, sc_pred2, sc_pred3, _, _, _ = sourceClassifier(target_data, training=False)

        logits1 = softmax(pred1)
        logits2 = softmax(pred2)
        logits3 = softmax(pred3)

        weights_sum = logits1[:, 1] * alpha[0] + logits2[:, 1] * alpha[1] + logits3[:, 1] * alpha[2]

        w1 = logits1[:, 1] * alpha[0] / weights_sum
        w2 = logits2[:, 1] * alpha[1] / weights_sum
        w3 = logits3[:, 1] * alpha[2] / weights_sum

        optimizer2.zero_grad()

        target_feature = model2(target_data)
        pred_target = model1(None, training=False, encoding=target_feature)

        _, _, _, feature1, feature2, feature3 = model1(target_data, training=False)

        loss = 0.0

        loss_value_tw = loss_tw(feature1, target_feature, w1, i-1, mark=1)
        loss_value_tw += loss_tw(feature2, target_feature, w2, i-1, mark=2)
        loss_value_tw += loss_tw(feature3, target_feature, w3, i-1, mark=3)
        loss_value_tw /= 3

        loss_value_distill = loss_distill(w1, w2, w3, sc_pred1, sc_pred2, sc_pred3, pred_target)
        loss_value_entropy = loss_entropy(pred_target)

        loss_value_de = loss_de(i, loss_value_distill, loss_value_entropy, m=0.01)

        loss = loss_value_tw + loss_value_de

        loss.backward()
        optimizer2.step()

        if i % log_interval == 0:
              print('Train target iter: {} [({:.0f}%)]\tT->WLoss: {:.6f}\tLoss_distill: {:.6f}\tLoss_entropy: {:.6f}\tLoss_de: {:.6f}\tLoss: {:.6f}'.format(
                  i, 100. * i / iteration, loss_value_tw.item(), loss_value_distill.item(), loss_value_entropy.item(), loss_value_de.item(), loss.item()))
    
    return model1, model2

def test(mode11, model2):
    model1.eval()
    model2.eval()

    test_loss = 0
    correct = 0
    correct1 = 0
    correct2 = 0
    correct3 = 0

    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.reshape(-1).cuda()
            data, target = Variable(data), Variable(target)
            feature = model2(data)

            pred = model1(None, training=False, encoding=feature)
            pred = pred.data.max(1)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        print(target_name, '\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
            correct, len(target_test_loader.dataset),
            100. * correct / len(target_test_loader.dataset)))
        print('\nsource1 accnum {}, source2 accnum {}，source3 accnum {}'.format(correct1, correct2, correct3))

    return correct

if __name__ == '__main__':
    sourceClassifier = SourceClassifer(n_classes=class_num)
    sourceClassifier.load_state_dict(torch.load("./results/source_classifer_ACP"))
    for param in sourceClassifier.parameters():
        param.requires_grad = False

    domainclassifier = DomainClassifier(sourceClassifier, n_classes=2)
    domainclassifier.load_state_dict(torch.load("./results/domain_classifer_ACP"))

    model1 = Targetclassifier(sourceClassifier)
    model2 = TargetEncoder()

    model1.load_state_dict(torch.load("./results/target_classifer_ACP"))
    model2.load_state_dict(torch.load("./results/target_encoder_ACP"))
    
    best_model_wts1 = copy.deepcopy(model1.state_dict())
    best_model_wts2 = copy.deepcopy(model2.state_dict())

    if cuda:
        model1.cuda()
        model2.cuda()
        domainclassifier.cuda()
    correct = 0
    for _ in range(epoch):
        model1, model2 = train(model1, model2, domainclassifier)
        t_correct = test(model1, model2)
        if t_correct > correct:
            correct = t_correct
            best_model_wts = copy.deepcopy(model1.state_dict())
        print(source1_name, source2_name, source3_name, "with", target_name, "%s max correct:" % target_name, correct.item(), "\n")
    best_model_wts2 = copy.deepcopy(model2.state_dict())
    torch.save(best_model_wts1, f"./results/target_classifer_ACP_new")
    torch.save(best_model_wts2, f"./results/target_encoder_ACP_new") 

# Final testing

In [ ]:
batch_size = 16
cuda = True
seed = 8
log_interval = 20
class_num = 65
root_path = "./Dataset/"
target_name = "Real World"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

def test(mode11, model2):
    model1.eval()
    model2.eval()

    correct = 0
    correct1 = 0
    correct2 = 0
    correct3 = 0

    target_test_loader = load_testing(root_path, target_name, batch_size, kwargs)

    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.reshape(-1).cuda()
            data, target = Variable(data), Variable(target)
            feature = model2(data)

            pred = model1(None, training=False, encoding=feature)
            pred = pred.data.max(1)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        print(target_name, '\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
            correct, len(target_test_loader.dataset),
            100. * correct / len(target_test_loader.dataset)))
        print('\nsource1 accnum {}, source2 accnum {}，source3 accnum {}'.format(correct1, correct2, correct3))

    return correct

if __name__ == '__main__':

    sourceClassifier = SourceClassifer(n_classes=class_num)
    sourceClassifier.load_state_dict(torch.load("./results/source_classifer_ACP"))
    for param in sourceClassifier.ResNet50.parameters():
        param.requires_grad = False

    model1 = Targetclassifier(sourceClassifier)
    model2 = TargetEncoder()

    model1.load_state_dict(torch.load("./results/target_classifer_ACP_tw"))
    model2.load_state_dict(torch.load("./results/target_encoder_ACP_tw"))

    if cuda:
        model1.cuda()
        model2.cuda()
    correct = 0

    t_correct = test(model1, model2)
    
    print(source1_name, source2_name, source3_name, "with", target_name, "%s max correct:" % target_name, t_correct.item(), "\n")
